In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import re
import uuid

from joblib import dump, load

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import balanced_accuracy_score

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from unidecode import unidecode

In [2]:
spanish_stemmer = SnowballStemmer('spanish')
portuguese_stemmer = SnowballStemmer('portuguese')

def spanish_stemmer_tokenizer(doc):
    tokens = word_tokenize(doc)
    return [spanish_stemmer.stem(token) for token in tokens]

def portuguese_stemmer_tokenizer(doc):
    tokens = word_tokenize(doc)
    return [portuguese_stemmer.stem(token) for token in tokens]


def my_preprocessor(doc):
    return re.sub('[0-9¡¨ª®°´·º»½¿' + string.punctuation + ']', '', unidecode(doc)).lower()

In [3]:
spanish_vectorizer = load('models/spanish_vectorizer_0495c1663b1442518c1282fa490a8be2.joblib')
spanish_classifier = load('models/spanish_classifier_0495c1663b1442518c1282fa490a8be2.joblib')

portuguese_vectorizer = load('models/portuguese_vectorizer_cb7e1220a901413aa002e26667cf29e6.joblib')
portuguese_classifier = load('models/portuguese_classifier_cb7e1220a901413aa002e26667cf29e6.joblib')

In [4]:
test = pd.read_csv('data/test.csv')
test.head()

,id,title,language
0,0,Kit Maternidade Bolsa-mala Baby/bebe Vinho Men...,portuguese
1,1,Trocador De Fraldas Fisher Price Feminino Rosa...,portuguese
2,2,Motor Ventoinha - Fiat Idea / Palio 1.8 - A 04...,portuguese
3,3,Amortecedor Mola Batente D Dir New Civic 14 - ...,portuguese
4,4,Cadeirinha De Carro Bebê Princesa Princess 9 A...,portuguese


In [5]:
test_portuguese = test[test.language=='portuguese']
test_spanish = test[test.language=='spanish']

In [6]:
test_features_spanish = spanish_vectorizer.transform(test_spanish['title'])
test_features_portuguese = portuguese_vectorizer.transform(test_portuguese['title'])

/home/david/miniconda3/envs/ml-challenge/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['estuvi', 'hubi', 'tuvi'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/david/miniconda3/envs/ml-challenge/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['dep'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [7]:
predictions_spanish = spanish_classifier.predict(test_features_spanish)
predictions_portuguese = portuguese_classifier.predict(test_features_portuguese)

df_spanish = pd.DataFrame(data={'id': test_spanish.index, 'category': predictions_spanish})
df_portuguese = pd.DataFrame(data={'id': test_portuguese.index, 'category': predictions_portuguese})
df_spanish.set_index('id', inplace=True)
df_portuguese.set_index('id', inplace=True)
df_spanish.head()


,category
id,
9,HARD_DRIVES_AND_SSDS
10,MEAT_GRINDERS
14,CROWBARS
15,HONEY
19,TURNTABLES


In [8]:
df_portuguese.head()

,category
id,
0,DIAPER_BAGS
1,BABY_CHANGING_PADS
2,ENGINE_COOLING_FAN_MOTORS
3,AUTOMOTIVE_SHOCK_ABSORBER_BUMP_STOPS
4,BABY_CAR_SEATS


In [9]:
df = df_spanish.append(df_portuguese)
df.sort_index(inplace=True)
df.head()

,category
id,
0,DIAPER_BAGS
1,BABY_CHANGING_PADS
2,ENGINE_COOLING_FAN_MOTORS
3,AUTOMOTIVE_SHOCK_ABSORBER_BUMP_STOPS
4,BABY_CAR_SEATS


In [10]:
df.to_csv('output/submission_7.csv')